In [1]:
import pandas as pd
import numpy as np

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [5]:
df=pd.read_csv(r"D:\100 Days machine learning\Column_Transformer day 28\covid_toy.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [43]:
df['city'].nunique()

4

#### fever has 10 null values need to fix that

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.2,random_state=0)

In [11]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [12]:
X_test

,age,gender,fever,cough,city
26,19,Female,100.0,Mild,Kolkata
86,25,Male,104.0,Mild,Bangalore
2,42,Male,101.0,Mild,Delhi
55,81,Female,101.0,Mild,Mumbai
75,5,Male,102.0,Mild,Kolkata
93,27,Male,100.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata
73,34,Male,98.0,Strong,Kolkata
54,60,Female,99.0,Mild,Mumbai
95,12,Female,104.0,Mild,Bangalore


# Aam Zindagi

In [15]:
#adding simple imputer to fever section to remove null value with Mean of the column
si=SimpleImputer()



In [17]:
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])


#all the null value filled by mean

## Ordinalencoding -> cough

In [29]:

oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])

## OneHotEncoding -> gender,city

In [33]:
ohe = OneHotEncoder(drop='first',sparse_output=False)

In [44]:
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])

In [45]:
X_train_gender_city.shape

(80, 4)

### Now will extract age

In [49]:
X_train_age=X_train.drop(columns=['gender','fever','cough','city'],axis=1).values
X_test_age=X_test.drop(columns=['gender','fever','cough','city'],axis=1).values


In [51]:
X_train_age.shape


(80, 1)

In [52]:
X_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

In [53]:
X_test_transformed=np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

In [55]:
X_train_transformed.shape

(80, 7)

# Now Mentos Zindagi

In [56]:
from sklearn.compose import ColumnTransformer

In [58]:
transformer= ColumnTransformer(transformers=[
('tnf1',SimpleImputer(),['fever']),
('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])

                               
],remainder='passthrough')

In [60]:
transformer.fit_transform(X_train).shape

(80, 7)

In [61]:
transformer.fit_transform(X_test).shape

(20, 7)